In [173]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
import json, re
from pathlib import Path
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

In [174]:
# Carregar o arquivo CSV
df_times = pd.read_excel("times.xlsx")

# Ver os dados carregados
display(df_times.head())

,Nome
0,A Lenda Super Vascão f.c
1,A Lenda Super Vasco F.c
2,Analove10 ITAQUI GRANDE!!
3,BordonFC
4,BORGES ITAQUI F.C.


In [175]:
# 👉 ajuste o caminho do HTML salvo
HTML_PATH = Path("pagina_liga_seria_A.html")

# carrega html
html = HTML_PATH.read_text(encoding="utf-8", errors="ignore")
soup = BeautifulSoup(html, "html.parser")

# 1) seletor principal dessa página
nomes = [el.get_text(strip=True) for el in soup.select("p.card-participantes-pontos-corridos__time")]

# 2) fallbacks (caso salve outra variante da página)
if not nomes:
    nomes = [el.get_text(strip=True) for el in soup.select("p.nome-time")]
if not nomes:
    nomes = [(a.get("title") or a.get_text(strip=True)).strip()
             for a in soup.select("a.link-perfil-time") if (a.get("title") or a.get_text(strip=True))]

# 3) fallback final via regex (se o HTML veio “achatado”)
if not nomes:
    for m in re.finditer(r'class="card-participantes-pontos-corridos__time"\s*>\s*([^<]+)<', html, re.I):
        nomes.append(m.group(1).strip())

# dedup + ordena alfabética
visto, times = set(), []
for n in nomes:
    n = re.sub(r"\s+", " ", n).strip()
    if n and n not in visto:
        visto.add(n); times.append(n)
times.sort(key=str.casefold)

print(f"{len(times)} times encontrados")
print(times)

# snippet JS ao lado do HTML (se quiser usar no site)
OUT_JS = HTML_PATH.with_name("participantesLiga_serie_A.js")
OUT_JS.write_text("window.participantesLiga = " + json.dumps(times, ensure_ascii=False, indent=2) + ";\n",
                  encoding="utf-8")
print(f"Snippet salvo em: {OUT_JS}")


20 times encontrados
['A Lenda Super Vasco F.c', 'Analove10 ITAQUI GRANDE!!', 'BORGES ITAQUI F.C.', 'cartola scheuer', 'DM Studio', 'Dom Camillo68', 'FBC Colorado', 'Fedato Futebol Clube', 'Gremiomaniasm', 'lsauer fc', 'MauHumor F.C.', 'pura bucha /botafogo', 'RHANKA DENTY FC25', 'Rolo Compressor ZN', 'seralex', 'Tatols Beants F.C', 'TEAM LOPES 99', 'Texas Club 2025', 'TIGRE LEON', 'Time do S.A.P.O']
Snippet salvo em: participantesLiga_serie_A.js


In [176]:
JS_PATH = Path("participantesLiga_serie_A.js")  # ajuste o caminho se estiver em outra pasta

# extrai o array do snippet: window.participantesLiga = [ ... ];
m = re.search(r'window\.participantesLiga\s*=\s*(\[[\s\S]*?\])\s*;', JS_PATH.read_text(encoding="utf-8"))
if not m:
    raise ValueError(f"Não consegui achar a lista dentro de {JS_PATH}")

nomes_times = json.loads(m.group(1))

# (opcional) dedup + ordena
nomes_times = sorted(dict.fromkeys(nomes_times), key=str.casefold)

print(len(nomes_times), "times")
nomes_times

20 times


['A Lenda Super Vasco F.c',
 'Analove10 ITAQUI GRANDE!!',
 'BORGES ITAQUI F.C.',
 'cartola scheuer',
 'DM Studio',
 'Dom Camillo68',
 'FBC Colorado',
 'Fedato Futebol Clube',
 'Gremiomaniasm',
 'lsauer fc',
 'MauHumor F.C.',
 'pura bucha /botafogo',
 'RHANKA DENTY FC25',
 'Rolo Compressor ZN',
 'seralex',
 'Tatols Beants F.C',
 'TEAM LOPES 99',
 'Texas Club 2025',
 'TIGRE LEON',
 'Time do S.A.P.O']

### Buscar IDs dos times no Cartola

In [177]:
# Dicionário para armazenar os IDs dos times
ids_times = {}

# Função robusta para buscar ID do time por nome
from difflib import get_close_matches

def buscar_id_time(nome_time):
    try:
        times = api.times(query=nome_time)

        # 🛡️ Proteção caso a resposta seja uma string (ex: erro HTML ou mensagem)
        if not isinstance(times, list):
            print(f"⚠️ Resposta inesperada para '{nome_time}':", times)
            return None

        nomes_api = [time.nome for time in times]

        # Comparação flexível (removendo acentos, pontos, etc)
        nome_base = nome_time.lower().replace(".", "").replace("fc", "").replace("f.c", "").strip()
        nomes_api_base = [n.lower().replace(".", "").replace("fc", "").replace("f.c", "").strip() for n in nomes_api]

        nome_proximo = get_close_matches(nome_base, nomes_api_base, n=1, cutoff=0.6)

        if nome_proximo:
            idx = nomes_api_base.index(nome_proximo[0])
            return times[idx].id

    except Exception as e:
        print(f"❌ Erro ao buscar ID para o time '{nome_time}': {e}")
    return None


# Buscar IDs automaticamente, sem sobrescrever os que já estiverem no dicionário
nao_encontrados = []

for nome in nomes_times:
    if nome not in ids_times:  # Protege os manuais
        time_id = buscar_id_time(nome)
        if time_id:
            ids_times[nome] = time_id
        else:
            nao_encontrados.append(nome)

# Mostrar resultado
print("IDs encontrados:", len(ids_times))
print("Times não encontrados automaticamente:", nao_encontrados)

IDs encontrados: 20
Times não encontrados automaticamente: []


### Gerar DataFrame com nome do time, ID e URL

In [178]:
# Gerar DataFrame com nome do time, ID e URL
df_urls = pd.DataFrame([
    {"Nome do Time": nome, "ID do Time": time_id, "Link do Time": f"https://cartola.globo.com/#!/time/{time_id}"}
    for nome, time_id in ids_times.items()
])

# Caminho e nome do arquivo Excel
caminho_excel = "links_times_cartola_liga_serie_A.xlsx"

# Salvar o DataFrame em Excel
df_urls.to_excel(caminho_excel, index=False)

print(f"✅ Arquivo salvo com sucesso: {caminho_excel}")


# Exibir como tabela
display(df_urls.head(30))

✅ Arquivo salvo com sucesso: links_times_cartola_liga_serie_A.xlsx


,Nome do Time,ID do Time,Link do Time
0,A Lenda Super Vasco F.c,117598,https://cartola.globo.com/#!/time/117598
1,Analove10 ITAQUI GRANDE!!,44574236,https://cartola.globo.com/#!/time/44574236
2,BORGES ITAQUI F.C.,3914981,https://cartola.globo.com/#!/time/3914981
3,cartola scheuer,3851966,https://cartola.globo.com/#!/time/3851966
4,DM Studio,387186,https://cartola.globo.com/#!/time/387186
5,Dom Camillo68,20696550,https://cartola.globo.com/#!/time/20696550
6,FBC Colorado,186283,https://cartola.globo.com/#!/time/186283
7,Fedato Futebol Clube,18642587,https://cartola.globo.com/#!/time/18642587
8,Gremiomaniasm,528730,https://cartola.globo.com/#!/time/528730
9,lsauer fc,44810918,https://cartola.globo.com/#!/time/44810918


### Gerar o dicionário ID -> Nome do Time

In [179]:
# Gerar o dicionário ID -> Nome do Time
nomes_por_id = dict(zip(df_urls["ID do Time"], df_urls["Nome do Time"]))

# Mostrar parte do dicionário
dict(list(nomes_por_id.items())[:5])

{117598: 'A Lenda Super Vasco F.c',
 44574236: 'Analove10 ITAQUI GRANDE!!',
 3914981: 'BORGES ITAQUI F.C.',
 3851966: 'cartola scheuer',
 387186: 'DM Studio'}

In [180]:
# ===============================
#  LIGA (2º TURNO) COM PARCIAIS DA RODADA EM ANDAMENTO
# ===============================
import time, json, requests, pandas as pd

# -------- CONFIG --------
TURNO_INICIO   = 20      # início do 2º turno
TOTAL_RODADAS  = 19      # quantidade de rodadas no 2º turno (20..38)
TURNO_FIM      = TURNO_INICIO + TOTAL_RODADAS - 1  # = 38
CAP_MULT       = 1.5     # multiplicador do capitão nas parciais
SLEEP_REQ      = 0.9     # pausa anti-rate-limit
MAX_RETRIES    = 3

# -------- CartolaFCError (fallback) --------
try:
    from cartolafc import CartolaFCError as _CartolaErr
except Exception:
    _CartolaErr = Exception

# -------- Sessão HTTP (para endpoints públicos) --------
sess = requests.Session()
sess.headers.update({
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json, text/plain, */*",
})

def http_status_e_rodada():
    """Retorna (status_mercado, rodada_http) via endpoint público."""
    r = sess.get("https://api.cartola.globo.com/mercado/status", timeout=20)
    r.raise_for_status()
    d = r.json()
    return int(d.get("status_mercado", 0)), int(d.get("rodada_atual", 0))

def get_parciais():
    r = sess.get("https://api.cartola.globo.com/atletas/pontuados", timeout=30)
    r.raise_for_status()
    return r.json().get("atletas", {})

def get_escalacao_por_rodada_http_dbg(time_id: int, rodada: int, max_retries=MAX_RETRIES):
    """
    Busca a escalação travada na rodada; se não vier, cai para 'última escalação'.
    Retorna: (atletas:list, capitao_id:int|None, http_status:int|None, motivo:str|None)
    """
    urls = [
        f"https://api.cartola.globo.com/time/id/{time_id}/{rodada}",
        f"https://api.cartola.globo.com/time/id/{time_id}",
    ]
    last_err = None
    for url in urls:
        wait = 0.7
        for _ in range(max_retries):
            try:
                r = sess.get(url, timeout=30)
                st = r.status_code
                if st == 200:
                    d = r.json()
                    atletas = d.get("atletas") or []
                    return atletas, d.get("capitao_id"), st, (None if atletas else "sem_atletas")
                if st in (403, 429, 503):
                    time.sleep(wait); wait *= 1.8; continue
                last_err = f"status={st}"
                break
            except Exception as e:
                last_err = f"exc:{type(e).__name__}"
                time.sleep(wait); wait *= 1.8
    return [], None, None, last_err

# -------- Regras de começo --------
def campeonato_comecou(api, ids_times):
    """
    Considera que o 2º turno começou se:
      - mercado indica rodada_atual >= TURNO_INICIO, e
      - há pelo menos uma pontuação não-nula na rodada TURNO_INICIO para algum time.
    """
    try:
        rodada_atual = getattr(api.mercado(), "rodada_atual", None)
    except _CartolaErr:
        rodada_atual = None

    if rodada_atual is None or rodada_atual < TURNO_INICIO:
        return False

    for time_id in ids_times.values():
        try:
            p = api.time(time_id=time_id, rodada=TURNO_INICIO).ultima_pontuacao
            if p is not None:
                return True
        except _CartolaErr:
            continue
    return False

def obter_pontuacao_por_rodada(api, time_id, rodada_atual):
    """
    Busca apenas rodadas concluídas: [TURNO_INICIO, rodada_atual-1].
    """
    pontuacoes = {}
    # fim exclusivo (rodada_atual não encerrada)
    fim_exclusivo = min(max(rodada_atual, TURNO_INICIO), TURNO_FIM + 1)
    for rodada in range(TURNO_INICIO, fim_exclusivo):
        try:
            time_r = api.time(time_id=time_id, rodada=rodada)
            pontuacoes[rodada] = time_r.ultima_pontuacao
        except _CartolaErr as e:
            print(f"⚠️ Erro ao acessar rodada {rodada} (time {time_id}): {e}")
            pontuacoes[rodada] = None
    return pontuacoes

def gerar_df_pontuacoes(api, ids_times):
    # 1) Colunas do turno (20..38)
    colunas_turno = [f"Rodada {i}" for i in range(TURNO_INICIO, TURNO_FIM + 1)]

    # 2) Rodada atual via lib + status via HTTP
    try:
        rodada_atual_api = api.mercado().rodada_atual
    except _CartolaErr:
        rodada_atual_api = TURNO_INICIO
    status_http, rodada_http = http_status_e_rodada()
    rodada_ref = rodada_http if rodada_http else rodada_atual_api
    print(f"Status HTTP={status_http} | rodada_http={rodada_http} | rodada_api={rodada_atual_api} | usando rodada_ref={rodada_ref}")

    # 3) Se turno ainda não começou, placeholders 0
    if not campeonato_comecou(api, ids_times):
        print("📌 2º Turno ainda não começou. Criando estrutura com placeholders (0).")
        return pd.DataFrame(0, index=list(ids_times.keys()), columns=colunas_turno)

    # 4) Monta DF com rodadas ENCERRADAS
    dados = {}
    for nome, time_id in ids_times.items():
        pontuacoes = obter_pontuacao_por_rodada(api, int(time_id), rodada_atual_api)
        s = pd.Series({f"Rodada {k}": v for k, v in pontuacoes.items()})
        dados[nome] = s

    df = pd.DataFrame.from_dict(dados, orient="index")
    df = df.reindex(columns=colunas_turno, fill_value=0).apply(pd.to_numeric, errors="coerce").fillna(0.0)

    # 5) Injeta PARCIAIS na rodada em andamento (se status==2 e dentro do turno)
    if status_http == 2 and TURNO_INICIO <= rodada_ref <= TURNO_FIM:
        col_atual = f"Rodada {rodada_ref}"
        print(f"🟡 Rodada {rodada_ref} em andamento — aplicando parciais…")
        parciais_map = get_parciais()
        if not parciais_map:
            print("⚠️ Endpoint de parciais vazio no momento. Mantendo valores anteriores.")
        else:
            atualizados, rel = 0, []
            for nome_time, time_id in ids_times.items():
                try:
                    atletas, capitao_id, http_st, motivo = get_escalacao_por_rodada_http_dbg(int(time_id), rodada_ref)

                    # Fallback via lib se vier "sem atletas"
                    if not atletas and (motivo in ("sem_atletas", "sem_escalacao", None)):
                        try:
                            t = api.time(time_id=int(time_id), rodada=rodada_ref)
                            atletas = [{"atleta_id": a.atleta_id} for a in (getattr(t, "atletas", None) or [])]
                            if not capitao_id:
                                capitao_id = getattr(t, "capitao_id", None)
                            http_st = http_st or 200
                        except Exception:
                            pass

                    total, matched = 0.0, 0
                    for a in atletas or []:
                        aid = str(a.get("atleta_id"))
                        if aid in parciais_map:
                            p = float(parciais_map[aid].get("pontuacao", 0) or 0)
                            if capitao_id and a.get("atleta_id") == capitao_id:
                                p *= CAP_MULT
                            total += p; matched += 1

                    df.loc[nome_time, col_atual] = round(total, 2)
                    atualizados += 1
                    rel.append((nome_time, http_st or 200, len(atletas or []), matched, round(total, 2)))
                    time.sleep(SLEEP_REQ)
                except Exception as e:
                    rel.append((nome_time, None, 0, 0, 0.0))

            print(f"✅ Times atualizados com parciais: {atualizados}/{len(ids_times)} na coluna '{col_atual}'")
            for nm, st, n_atl, n_ok, tot in rel[:12]:
                print(f" • {nm:25} | HTTP={st} | atletas={n_atl:2d} | com_parcial={n_ok:2d} | total={tot:6.2f}")
    else:
        print("ℹ️ Rodada não está em andamento (status != 2) ou fora do intervalo do turno — sem parciais.")

    # 6) (Opcional) Linha 'Lider_Rodada' quando houver alguma coluna com dado > 0
    try:
        cols_com_dado = [c for c in colunas_turno if df[c].notna().any()]
        if cols_com_dado and df[cols_com_dado].sum(numeric_only=True).sum() != 0:
            df.loc['Lider_Rodada'] = df[cols_com_dado].idxmax(axis=0)
    except Exception as e:
        print("⚠️ Não foi possível recalcular 'Lider_Rodada':", e)

    return df

# === Exemplo de uso ===
# ids_times = {"Nome do Time": 12345, ...}
df_pontuacoes = gerar_df_pontuacoes(api, ids_times)

# Visualização
try:
    display(df_pontuacoes.T)
except Exception:
    print(df_pontuacoes.T.head(20))


Status HTTP=2 | rodada_http=20 | rodada_api=20 | usando rodada_ref=20
🟡 Rodada 20 em andamento — aplicando parciais…
✅ Times atualizados com parciais: 20/20 na coluna 'Rodada 20'
 • A Lenda Super Vasco F.c   | HTTP=200 | atletas=12 | com_parcial=10 | total= 56.53
 • Analove10 ITAQUI GRANDE!! | HTTP=200 | atletas=12 | com_parcial= 9 | total= 47.81
 • BORGES ITAQUI F.C.        | HTTP=200 | atletas=12 | com_parcial= 9 | total= 53.26
 • cartola scheuer           | HTTP=200 | atletas=12 | com_parcial=10 | total= 47.36
 • DM Studio                 | HTTP=200 | atletas=12 | com_parcial=10 | total= 37.01
 • Dom Camillo68             | HTTP=200 | atletas=12 | com_parcial= 8 | total= 23.40
 • FBC Colorado              | HTTP=200 | atletas=12 | com_parcial= 8 | total= 38.36
 • Fedato Futebol Clube      | HTTP=200 | atletas=12 | com_parcial=10 | total= 47.36
 • Gremiomaniasm             | HTTP=200 | atletas=12 | com_parcial= 7 | total= 29.76
 • lsauer fc                 | HTTP=200 | atletas=12 | c

,A Lenda Super Vasco F.c,Analove10 ITAQUI GRANDE!!,BORGES ITAQUI F.C.,cartola scheuer,DM Studio,Dom Camillo68,FBC Colorado,Fedato Futebol Clube,Gremiomaniasm,lsauer fc,MauHumor F.C.,pura bucha /botafogo,RHANKA DENTY FC25,Rolo Compressor ZN,seralex,Tatols Beants F.C,TEAM LOPES 99,Texas Club 2025,TIGRE LEON,Time do S.A.P.O,Lider_Rodada
Rodada 20,56.53,47.81,53.26,47.36,37.01,23.40,38.36,47.36,29.76,31.76,37.81,52.81,52.33,40.21,45.26,55.71,36.96,53.66,29.86,59.85,Time do S.A.P.O
Rodada 21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rodada 22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rodada 23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rodada 24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rodada 25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rodada 26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rodada 27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rodada 28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rodada 29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Função para definir a classificação dos times

In [181]:
def classificacao_por_grupo(
    df_rodadas: pd.DataFrame,
    df_pontuacoes: pd.DataFrame,
    rodadas_encerradas: set[int] | None = None,
    rodada_parcial: int | None = None,
):
    """
    Gera classificação por grupo a partir de confrontos (df_rodadas) e pontuações (df_pontuacoes).

    - `rodadas_encerradas`: rodadas 100% fechadas (contam normalmente).
    - `rodada_parcial`: rodada em andamento; só conta confrontos se ambos tiverem número e
      pelo menos UM deles > 0 (evita empates 0x0 de placeholder).
    """

    # --- 0) Normalizações para casar chaves ---
    df_pontuacoes_times = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore').copy()
    if df_pontuacoes_times.index.dtype != "object":
        df_pontuacoes_times.index = df_pontuacoes_times.index.astype(str)
    df_pontuacoes_times.index = df_pontuacoes_times.index.str.strip()

    # colunas do tipo "Rodada N" -> string limpinha
    df_pontuacoes_times.columns = [str(c).strip() for c in df_pontuacoes_times.columns]

    # garante colunas esperadas no df_rodadas
    required_cols = {"Grupo", "Mandante_Nome", "Visitante_Nome", "Rodada"}
    faltando = required_cols - set(df_rodadas.columns)
    if faltando:
        raise ValueError(f"df_rodadas está sem as colunas: {sorted(faltando)}")

    # --- 1) Pré-inicializa times (zerados) por grupo, a partir dos confrontos ---
    estatisticas: dict[str, dict[str, dict]] = {}
    for _, r in df_rodadas.iterrows():
        grupo = str(r["Grupo"]).strip()
        mandante = str(r["Mandante_Nome"]).strip()
        visitante = str(r["Visitante_Nome"]).strip()

        if grupo not in estatisticas:
            estatisticas[grupo] = {}

        for time in (mandante, visitante):
            if time not in estatisticas[grupo]:
                estatisticas[grupo][time] = {
                    "Pontos": 0,
                    "Vitórias": 0,
                    "Empates": 0,
                    "Derrotas": 0,
                    "Total_Cartola": 0.0,
                    "Cartola_Sofrido": 0.0,
                }

    # helper seguro para ler pontuação do DF; None => não considerar
    def safe_pontuacao(nome_time: str, coluna_rodada: str):
        if (nome_time in df_pontuacoes_times.index) and (coluna_rodada in df_pontuacoes_times.columns):
            val = df_pontuacoes_times.at[nome_time, coluna_rodada]
            if pd.notnull(val):
                try:
                    return float(val)
                except Exception:
                    return None
        return None

    # se o caller não passar, por padrão considera todas as rodadas do df_rodadas como "encerradas"
    # (na prática, é melhor passar explicitamente de fora)
    if rodadas_encerradas is None:
        try:
            rodadas_encerradas = set(int(x) for x in df_rodadas["Rodada"].dropna().unique())
        except Exception:
            rodadas_encerradas = set()

    # --- 2) Processa confrontos conforme status da rodada ---
    for _, confronto in df_rodadas.iterrows():
        try:
            rodada = int(confronto["Rodada"])
        except Exception:
            # se vier string tipo "20", tenta converter; caso contrário, pula
            try:
                rodada = int(str(confronto["Rodada"]).strip())
            except Exception:
                continue

        mandante = str(confronto["Mandante_Nome"]).strip()
        visitante = str(confronto["Visitante_Nome"]).strip()
        grupo = str(confronto["Grupo"]).strip()
        coluna_rodada = f"Rodada {rodada}"

        # Rodada não elegível? (nem encerrada, nem a parcial atual) → ignora
        if (rodada not in rodadas_encerradas) and (rodada != rodada_parcial):
            continue

        pm = safe_pontuacao(mandante, coluna_rodada)
        pv = safe_pontuacao(visitante, coluna_rodada)

        # Regra para rodada PARCIAL: só conta se ambos têm número e pelo menos UM > 0
        if rodada == rodada_parcial:
            if pm is None or pv is None:
                continue
            if (pm == 0.0) and (pv == 0.0):
                continue  # evita empates 0x0 de placeholder

        # Regra para rodada ENCERRADA: exige números válidos para ambos
        if rodada in rodadas_encerradas:
            if pm is None or pv is None:
                continue

        # --- acumula totais de cartola ---
        estatisticas[grupo][mandante]["Total_Cartola"] += pm
        estatisticas[grupo][mandante]["Cartola_Sofrido"] += pv
        estatisticas[grupo][visitante]["Total_Cartola"] += pv
        estatisticas[grupo][visitante]["Cartola_Sofrido"] += pm

        # --- pontos/vit/emp/der ---
        if pm > pv:
            estatisticas[grupo][mandante]["Pontos"] += 3
            estatisticas[grupo][mandante]["Vitórias"] += 1
            estatisticas[grupo][visitante]["Derrotas"] += 1
        elif pm < pv:
            estatisticas[grupo][visitante]["Pontos"] += 3
            estatisticas[grupo][visitante]["Vitórias"] += 1
            estatisticas[grupo][mandante]["Derrotas"] += 1
        else:
            estatisticas[grupo][mandante]["Pontos"] += 1
            estatisticas[grupo][visitante]["Pontos"] += 1
            estatisticas[grupo][mandante]["Empates"] += 1
            estatisticas[grupo][visitante]["Empates"] += 1

    # --- 3) Monta DataFrame final (mesmo se ninguém contou) ---
    frames = []
    for grupo, times in estatisticas.items():
        if not times:
            continue
        df_g = pd.DataFrame({
            "Grupo": grupo,
            "Nome do Time": list(times.keys()),
            "Pontos": [stats["Pontos"] for stats in times.values()],
            "Vitórias": [stats["Vitórias"] for stats in times.values()],
            "Empates": [stats["Empates"] for stats in times.values()],
            "Derrotas": [stats["Derrotas"] for stats in times.values()],
            "Total Cartola": [stats["Total_Cartola"] for stats in times.values()],
            "Cartola Sofrido": [stats["Cartola_Sofrido"] for stats in times.values()],
        })
        df_g["Saldo Cartola"] = df_g["Total Cartola"] - df_g["Cartola Sofrido"]
        frames.append(df_g)

    if not frames:
        cols = ["Grupo","Nome do Time","Pontos","Vitórias","Empates","Derrotas",
                "Total Cartola","Cartola Sofrido","Saldo Cartola","Posição"]
        return pd.DataFrame(columns=cols), {}

    df_resultado = pd.concat(frames, ignore_index=True)

    # ordenação (desempate por Total/Saldo e depois Nome)
    df_resultado = df_resultado.sort_values(
        by=["Grupo", "Pontos", "Total Cartola", "Saldo Cartola", "Nome do Time"],
        ascending=[True, False, False, False, True]
    ).reset_index(drop=True)

    df_resultado["Posição"] = df_resultado.groupby("Grupo").cumcount() + 1

    df_resultado_por_grupo = {
        g: df_resultado[df_resultado["Grupo"] == g].copy()
        for g in df_resultado["Grupo"].unique()
    }

    return df_resultado, df_resultado_por_grupo


In [182]:
# 1) Confrontos
df_confrontos = pd.read_csv("confrontos_serie_A.csv")

# 2) Renomeia colunas (se vierem como "Time A"/"Time B")
df_confrontos.rename(columns={
    "Time A": "Mandante_Nome",
    "Time B": "Visitante_Nome"
}, inplace=True)

# 3) Grupo fixo
df_confrontos["Grupo"] = "Série A"

# 4) Série A do 2º turno → converter Rodada 1..19 (UI) para 20..38 (Cartola)
TURNO_INICIO, TURNO_FIM = 20, 38
df_confrontos["Rodada"] = df_confrontos["Rodada"].astype(int)
if df_confrontos["Rodada"].max() <= 19:
    df_confrontos["Rodada"] = df_confrontos["Rodada"] + (TURNO_INICIO - 1)  # 1→20, 19→38

# 5) Status do mercado (para saber se a rodada corrente está em andamento)
import requests
def http_status_e_rodada():
    r = requests.get("https://api.cartola.globo.com/mercado/status", timeout=20)
    r.raise_for_status()
    d = r.json()
    return int(d.get("status_mercado", 0)), int(d.get("rodada_atual", 0))

status_http, rodada_http = http_status_e_rodada()   # status 2 = em andamento
rodada_http = int(rodada_http or TURNO_INICIO)

# 6) Parâmetros para a classificação
#    - rodada_parcial: somente se status==2 e dentro de 20..38
#    - rodadas_encerradas:
#         * com parcial → [20 .. rodada_parcial-1]
#         * sem parcial → [20 .. min(rodada_http, 38)] (inclusivo)
rodada_parcial = rodada_http if (status_http == 2 and TURNO_INICIO <= rodada_http <= TURNO_FIM) else None
if rodada_parcial:
    rodadas_encerradas = set(range(TURNO_INICIO, rodada_parcial))  # fim exclusivo
else:
    rodadas_encerradas = set(range(TURNO_INICIO, min(rodada_http, TURNO_FIM) + 1))  # fim inclusivo

print(f"[DEBUG] status={status_http} rodada_http={rodada_http} encerradas={sorted(rodadas_encerradas)} parcial={rodada_parcial}")

# 7) Alinha nomes do DF de pontuações
df_pontuacoes.index = df_pontuacoes.index.astype(str).str.strip()

# 8) Classificação (a função deve aceitar 'rodadas_encerradas' e 'rodada_parcial')
df_classificacao, _ = classificacao_por_grupo(
    df_rodadas=df_confrontos,
    df_pontuacoes=df_pontuacoes,
    rodadas_encerradas=rodadas_encerradas,
    rodada_parcial=rodada_parcial,
)

# 9) Visualiza
display(df_classificacao.head(30))


[DEBUG] status=2 rodada_http=20 encerradas=[] parcial=20


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
0,Série A,Time do S.A.P.O,3,1,0,0,59.85,23.40,36.45,1
1,Série A,A Lenda Super Vasco F.c,3,1,0,0,56.53,47.36,9.17,2
2,Série A,Tatols Beants F.C,3,1,0,0,55.71,37.81,17.90,3
3,Série A,Texas Club 2025,3,1,0,0,53.66,52.81,0.85,4
4,Série A,BORGES ITAQUI F.C.,3,1,0,0,53.26,47.81,5.45,5
5,Série A,RHANKA DENTY FC25,3,1,0,0,52.33,31.76,20.57,6
6,Série A,cartola scheuer,3,1,0,0,47.36,45.26,2.10,7
7,Série A,Rolo Compressor ZN,3,1,0,0,40.21,38.36,1.85,8
8,Série A,DM Studio,3,1,0,0,37.01,36.96,0.05,9
9,Série A,TIGRE LEON,3,1,0,0,29.86,29.76,0.10,10


In [183]:
# 1. Times únicos nos confrontos
times_confrontos = pd.unique(df_confrontos[["Mandante_Nome", "Visitante_Nome"]].values.ravel())

# 2. Times únicos na classificação final
times_classificados = df_classificacao["Nome do Time"].unique()

# 3. Ver quem está nos confrontos mas não foi classificado
faltando = set(times_confrontos) - set(times_classificados)

print("❌ Times que estão nos confrontos, mas faltam na classificação:")
print(faltando)

# df_pontuacoes.loc[["Time do S.A.P.O", "BORGES ITAQUI F.C."]]


❌ Times que estão nos confrontos, mas faltam na classificação:
set()


In [184]:
# === Série A — Exportadores (com parciais do 2º turno) ===
import pandas as pd, json, math

# -------------------------------------------
# 1) CLASSIFICAÇÃO -> classificacao_serie_A.js
# -------------------------------------------
# df_classificacao já está no formato das suas colunas "originais"
df_classificacao.to_csv("classificacao_serie_a.csv", index=False)

df_classificacao = pd.read_csv("classificacao_serie_a.csv")
df_classificacao.rename(columns={
    "Grupo": "grupo",
    "Nome do Time": "nome",
    "Pontos": "pontos",
    "Vitórias": "vitorias",
    "Empates": "empates",
    "Derrotas": "derrotas",
    "Total Cartola": "totalCartola",
    "Cartola Sofrido": "cartolaSofrido",
    "Saldo Cartola": "saldoCartola",
    "Posição": "posicao",
}, inplace=True)

classificacao_por_grupo = {}
for grupo, dados in df_classificacao.groupby("grupo"):
    classificacao_por_grupo[grupo] = dados.drop(columns="grupo").to_dict(orient="records")

with open("classificacao_serie_A.js", "w", encoding="utf-8") as f:
    f.write("const classificacaoSerieA = ")
    json.dump(classificacao_por_grupo, f, ensure_ascii=False, indent=2)
    f.write(";")
print("✅ classificacao_serie_A.js")


✅ classificacao_serie_A.js


In [185]:
# -------------------------------------------
# 2) CONFRONTOS -> confrontos_serie_A.js
# -------------------------------------------
df_confrontos = pd.read_csv("confrontos_serie_A.csv")
df_confrontos.columns = df_confrontos.columns.str.strip()
df_confrontos.rename(columns={
    "Rodada": "rodada",
    "Confronto": "confronto",
    "Time A": "mandante_nome",
    "Time B": "visitante_nome",
    "ID A": "mandante_id",
    "ID B": "visitante_id",
}, inplace=True)
df_confrontos["grupo"] = "Série A"

confrontos_formatado = []
for _, row in df_confrontos.iterrows():
    confrontos_formatado.append({
        "rodada": int(row["rodada"]),
        "confronto": int(row["confronto"]),
        "grupo": row["grupo"],
        "mandante": {"id": row["mandante_id"], "nome": row["mandante_nome"]},
        "visitante": {"id": row["visitante_id"], "nome": row["visitante_nome"]},
    })

with open("confrontos_serie_A.js", "w", encoding="utf-8") as f:
    f.write("const confrontosFase1 = ")
    json.dump(confrontos_formatado, f, ensure_ascii=False, indent=2)
    f.write(";")
print("✅ confrontos_serie_A.js")


✅ confrontos_serie_A.js


In [186]:
# -------------------------------------------
# 3) RESULTADOS (com parciais) -> resultados_serie_A.js
# -------------------------------------------
# df_pontuacoes: index = nome do time, colunas = 'Rodada 20'..'Rodada 38' (ou 1..19)
TURNO_INICIO = 20
TURNO_OFFSET = TURNO_INICIO - 1  # 19  (UI 1 => Cartola 20)

# normaliza df_pontuacoes
dfp = df_pontuacoes.copy()
dfp.index = dfp.index.astype(str).str.strip()
dfp.columns = [str(c) for c in dfp.columns]
dfp = dfp.apply(pd.to_numeric, errors="coerce")

def _get_ponto(team: str, rodada_ui: int):
    """Busca ponto do 'team' em 'rodada_ui' aceitando:
       - coluna 'Rodada {rodada_ui+19}' (2º turno),
       - ou 'Rodada {rodada_ui}' (caso tenha sido salvo assim)."""
    col20 = f"Rodada {rodada_ui + TURNO_OFFSET}"
    col01 = f"Rodada {rodada_ui}"
    v = None
    if team in dfp.index and col20 in dfp.columns:
        v = dfp.at[team, col20]
    if (v is None or pd.isna(v)) and team in dfp.index and col01 in dfp.columns:
        v = dfp.at[team, col01]
    if v is None or pd.isna(v):
        return None
    return float(v)

def gerar_resultados_serie_a(df_confrontos_renamed: pd.DataFrame) -> pd.DataFrame:
    out = []
    for _, row in df_confrontos_renamed.iterrows():
        rodada = int(row["rodada"])
        mandante = str(row["mandante_nome"]).strip()
        visitante = str(row["visitante_nome"]).strip()
        pm = _get_ponto(mandante, rodada)
        pv = _get_ponto(visitante, rodada)
        out.append({
            "grupo": row.get("grupo", "Série A"),
            "rodada": rodada,
            "mandante": {"nome": mandante, "pontos": None if pm is None else round(pm, 2)},
            "visitante": {"nome": visitante, "pontos": None if pv is None else round(pv, 2)},
        })
    return pd.DataFrame(out)

df_resultados = gerar_resultados_serie_a(df_confrontos)

with open("resultados_serie_A.js", "w", encoding="utf-8") as f:
    f.write("const resultadosFase1 = ")
    json.dump(df_resultados.to_dict(orient="records"), f, ensure_ascii=False, indent=2)
    f.write(";")
print("✅ resultados_serie_A.js (com parciais quando houver)")

# -------------------------------------------
# 4) PONTUAÇÕES POR RODADA (opcional, pro front) -> pontuacoes_por_rodada_serie_A.js
#    Gera chaves 'Rodada 1..19' a partir do df_pontuacoes
# -------------------------------------------
def df_to_ui_rounds(dfp: pd.DataFrame):
    mapa = {}
    for team in dfp.index:
        inner = {}
        for ui in range(1, 20):
            col20 = f"Rodada {ui + TURNO_OFFSET}"
            col01 = f"Rodada {ui}"
            v = None
            if col20 in dfp.columns:
                vv = dfp.at[team, col20]
                if pd.notna(vv):
                    v = float(vv)
            if v is None and col01 in dfp.columns:
                vv = dfp.at[team, col01]
                if pd.notna(vv):
                    v = float(vv)
            if v is not None:
                inner[f"Rodada {ui}"] = v
        if inner:
            mapa[team] = inner
    return mapa

with open("pontuacoes_por_rodada_serie_A.js", "w", encoding="utf-8") as f:
    f.write("window.pontuacoesPorRodada = ")
    json.dump(df_to_ui_rounds(dfp), f, ensure_ascii=False, indent=2)
    f.write(";")
print("✅ pontuacoes_por_rodada_serie_A.js")

print("🎉 Pronto. Recarregue a página com os 4 arquivos gerados incluídos antes do 'scripts/serie_A.js'.")


✅ resultados_serie_A.js (com parciais quando houver)
✅ pontuacoes_por_rodada_serie_A.js
🎉 Pronto. Recarregue a página com os 4 arquivos gerados incluídos antes do 'scripts/serie_A.js'.


In [187]:
# Carregar o arquivo CSV enviado
df_confrontos = pd.read_csv("confrontos_serie_A.csv")

display(df_confrontos.head())

,Rodada,Confronto,Time A,Time B,ID A,ID B
0,1,1,Rolo Compressor ZN,FBC Colorado,11,6
1,1,2,BORGES ITAQUI F.C.,Analove10 ITAQUI GRANDE!!,3,2
2,1,3,cartola scheuer,seralex,17,20
3,1,4,Texas Club 2025,pura bucha /botafogo,15,19
4,1,5,lsauer fc,RHANKA DENTY FC25,18,10
